### 1. Proposition: Full details (name and email) of people who are not in a sales role
**Functional specification:**
- Create a set of BusinessEntityIDs for all people from the Person.Person table.
- Create a second set of BusinessEntityIDs for people who are explicitly listed in the Sales.SalesPerson table.
- Use the EXCEPT operator to find the BusinessEntityIDs of people who are not salespeople.
- Join this resulting set with the Person.Person and Person.EmailAddress tables to retrieve the full name and email address.

In [ ]:
USE AdventureWorks2017;

WITH NonSalesPeople AS (
    SELECT BusinessEntityID FROM Person.Person
    EXCEPT
    SELECT BusinessEntityID FROM Sales.SalesPerson
)
SELECT
    p.FirstName,
    p.LastName,
    ea.EmailAddress
FROM Person.Person p
JOIN NonSalesPeople nsp ON p.BusinessEntityID = nsp.BusinessEntityID
JOIN Person.EmailAddress ea ON p.BusinessEntityID = ea.BusinessEntityID
ORDER BY p.LastName, p.FirstName;

### 2. Proposition: Products that have never been on a special offer, along with their list price

**Functional specification:**
- Create a set of all ProductIDs from the main product list.
- Create a second set of all unique ProductIDs that have appeared on a special offer.
- Use the EXCEPT operator to find the products that have never been on offer.
- Join this result with the Product table to retrieve the full product name and its list price, ordered by price descending.

In [ ]:
WITH NeverOnOffer AS (
    SELECT ProductID FROM Production.Product
    EXCEPT
    SELECT DISTINCT ProductID FROM Sales.SpecialOfferProduct
)
SELECT
    p.Name,
    p.ProductNumber,
    p.ListPrice
FROM Production.Product p
JOIN NeverOnOffer noo ON p.ProductID = noo.ProductID
WHERE p.ListPrice > 0
ORDER BY p.ListPrice DESC;

### 3. Proposition: Names of sales territories that have customers but no assigned salespersons

**Functional specification:**
- Generate a distinct set of TerritoryIDs for all territories that have customers.
- Generate a second distinct set of TerritoryIDs for all territories that have an assigned salesperson.
- Use the EXCEPT operator to find territories that are in the first set but not the second.
- Join this result with the SalesTerritory table to get the full name and country code of each territory.

In [ ]:
WITH TerritoriesWithoutSalesperson AS (
    SELECT DISTINCT TerritoryID FROM Sales.Customer WHERE TerritoryID IS NOT NULL
    EXCEPT
    SELECT DISTINCT TerritoryID FROM Sales.SalesPerson WHERE TerritoryID IS NOT NULL
)
SELECT
    st.Name,
    st.CountryRegionCode
FROM Sales.SalesTerritory st
JOIN TerritoriesWithoutSalesperson t ON st.TerritoryID = t.TerritoryID
ORDER BY st.Name;

### 4. Proposition: Full names of customers who have purchased products from both the 'Bikes' and 'Accessories' categories

**Functional specification:**
- Create a set of CustomerIDs who have purchased a product from the 'Bikes' category.
- Create a second set of CustomerIDs who have purchased a product from the 'Accessories' category.
- Use the INTERSECT operator to find the CustomerIDs present in both sets.
- Join this final set of IDs with the Person table to retrieve the full names of these customers.

In [ ]:
WITH IntersectingCustomers AS (
    SELECT soh.CustomerID
    FROM Sales.SalesOrderHeader AS soh
    JOIN Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
    JOIN Production.Product AS p ON sod.ProductID = p.ProductID
    JOIN Production.ProductSubcategory AS psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory AS pc ON psc.ProductCategoryID = pc.ProductCategoryID
    WHERE pc.Name = 'Bikes'
    INTERSECT
    SELECT soh.CustomerID
    FROM Sales.SalesOrderHeader AS soh
    JOIN Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
    JOIN Production.Product AS p ON sod.ProductID = p.ProductID
    JOIN Production.ProductSubcategory AS psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory AS pc ON psc.ProductCategoryID = pc.ProductCategoryID
    WHERE pc.Name = 'Accessories'
)
SELECT
    p.FirstName,
    p.LastName
FROM Person.Person p
JOIN IntersectingCustomers ic ON p.BusinessEntityID = ic.CustomerID
ORDER BY p.LastName, p.FirstName;

### 5. Proposition: Names of vendors who supply component parts but do not supply any finished bicycles

**Functional specification:**
- Generate a set of BusinessEntityIDs for vendors who supply component parts (FinishedGoodsFlag = 0).
- Generate a second set for vendors who supply any product that is a finished bicycle.
- Use the EXCEPT operator to subtract the second set from the first.
- Join the resulting vendor IDs with the Vendor table to get their names and credit ratings.

In [ ]:
WITH ComponentOnlyVendors AS (
    SELECT pv.BusinessEntityID
    FROM Purchasing.ProductVendor AS pv
    JOIN Production.Product AS p ON pv.ProductID = p.ProductID
    WHERE p.FinishedGoodsFlag = 0
    EXCEPT
    SELECT pv.BusinessEntityID
    FROM Purchasing.ProductVendor AS pv
    JOIN Production.Product AS p ON pv.ProductID = p.ProductID
    JOIN Production.ProductSubcategory AS psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory AS pc ON psc.ProductCategoryID = pc.ProductCategoryID
    WHERE p.FinishedGoodsFlag = 1 AND pc.Name = 'Bikes'
)
SELECT
    v.Name,
    v.CreditRating
FROM Purchasing.Vendor v
JOIN ComponentOnlyVendors cov ON v.BusinessEntityID = cov.BusinessEntityID
ORDER BY v.Name;

### 6. Proposition: Names of salespersons who had a quota in both Q1 and Q2 of 2013

**Functional specification:**
- Create a set of salesperson IDs who had a quota recorded in the first quarter of 2013.
- Create a second set of salesperson IDs who had a quota recorded in the second quarter of 2013.
- Use the INTERSECT operator to find the salespersons who appear in both sets.
- Join the result with the Person table to get the full name of each salesperson.

In [ ]:
WITH ConsistentSalespersons AS (
    SELECT BusinessEntityID
    FROM Sales.SalesPersonQuotaHistory
    WHERE QuotaDate >= '2013-01-01' AND QuotaDate < '2013-04-01'
    INTERSECT
    SELECT BusinessEntityID
    FROM Sales.SalesPersonQuotaHistory
    WHERE QuotaDate >= '2013-04-01' AND QuotaDate < '2013-07-01'
)
SELECT
    p.FirstName,
    p.LastName
FROM Person.Person p
JOIN ConsistentSalespersons cs ON p.BusinessEntityID = cs.BusinessEntityID
ORDER BY p.LastName, p.FirstName;

### 7. Proposition: Orders that use an address for both billing and shipping

**Functional specification:**
- Create a set of all unique address IDs used as a billing address.
- Create a second set of all unique address IDs used as a shipping address.
- Use the INTERSECT operator to find the common addresses.
- Join this result back to the SalesOrderHeader table to identify the specific orders and customers using these common addresses.

In [ ]:
WITH CommonAddresses AS (
    SELECT BillToAddressID AS AddressID FROM Sales.SalesOrderHeader
    INTERSECT
    SELECT ShipToAddressID AS AddressID FROM Sales.SalesOrderHeader
)
SELECT
    soh.SalesOrderID,
    soh.OrderDate,
    soh.CustomerID,
    ca.AddressID AS CommonAddressID
FROM Sales.SalesOrderHeader soh
JOIN CommonAddresses ca ON soh.BillToAddressID = ca.AddressID
ORDER BY soh.OrderDate;

### 8. Proposition: Names of products that are in inventory and have also been sold

**Functional specification:**
- Generate a set of all distinct ProductIDs from inventory.
- Generate a second set of all distinct ProductIDs from sales order details.
- Use the INTERSECT operator to find the common products.
- Join the result with the Product table to get the product names and their color.

In [ ]:
WITH SoldInventoryProducts AS (
    SELECT DISTINCT ProductID FROM Production.ProductInventory
    INTERSECT
    SELECT DISTINCT ProductID FROM Sales.SalesOrderDetail
)
SELECT
    p.Name,
    p.Color,
    p.ListPrice
FROM Production.Product p
JOIN SoldInventoryProducts sip ON p.ProductID = sip.ProductID
ORDER BY p.Name;

### 9. Proposition: A combined list of people who are either Sales Staff or Vendor Contacts

**Functional specification:**
- Create a set of full names and a role type for all people who are in the SalesPerson table.
- Create a second set of full names and a role type for all people who are primary contacts for a vendor.
- Use the UNION operator to combine these two sets into a single list of unique people and their roles.

In [ ]:
SELECT
    p.FirstName,
    p.LastName,
    'Sales Person' AS PersonRole
FROM Person.Person p
JOIN Sales.SalesPerson sp ON p.BusinessEntityID = sp.BusinessEntityID
UNION
SELECT
    p.FirstName,
    p.LastName,
    'Vendor Contact' AS PersonRole
FROM Person.Person p
JOIN Person.BusinessEntityContact bec ON p.BusinessEntityID = bec.PersonID
JOIN Purchasing.Vendor v ON bec.BusinessEntityID = v.BusinessEntityID
ORDER BY PersonRole, p.LastName;

### 10. Proposition: Names of currencies used in prior years but not in the current year

**Functional specification:**
- Create a set of all distinct currency codes used in orders placed before the current calendar year.
- Create a second set of all distinct currency codes used in orders placed during the current calendar year.
- Use the EXCEPT operator to find the historic-only currencies.
- Join this result with the Currency table to display the full name of these currencies.

In [ ]:
WITH HistoricOnlyCodes AS (
    SELECT DISTINCT scr.ToCurrencyCode AS CurrencyCode
    FROM Sales.SalesOrderHeader soh
    JOIN Sales.CurrencyRate scr ON soh.CurrencyRateID = scr.CurrencyRateID
    WHERE YEAR(soh.OrderDate) < YEAR(GETDATE())
    EXCEPT
    SELECT DISTINCT scr.ToCurrencyCode AS CurrencyCode
    FROM Sales.SalesOrderHeader soh
    JOIN Sales.CurrencyRate scr ON soh.CurrencyRateID = scr.CurrencyRateID
    WHERE YEAR(soh.OrderDate) = YEAR(GETDATE())
)
SELECT
    c.Name
FROM Sales.Currency c
JOIN HistoricOnlyCodes h ON c.CurrencyCode = h.CurrencyCode
ORDER BY c.Name;